Copyright 2019 The TensorFlow Authors.

```
@title Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License.
You may obtain a copy of the License at
https://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
```

# Mô hình Transformer để hiểu ngôn ngữ

Đây hướng dẫn huấn luyện một <a href="https://arxiv.org/abs/1706.03762" class="external">mô hình Transformer</a> để dịch một [tập dữ liệu Bồ Đào Nha sang tiếng Anh](https://www.tensorflow.org/datasets/catalog/ted_hrlr_translate#ted_hrlr_translatept_to_en). Đây là một ví dụ nâng cao mà giả định kiến thức của [sinh văn bản](https://www.tensorflow.org/text/tutorials/text_generation) và [chú ý](https://www.tensorflow.org/text/tutorials/nmt_with_attention).

Ý tưởng cốt lõi đằng sau mô hình Transformer là *tự chú ý* - khả năng tham gia vào các vị trí khác nhau của chuỗi đầu vào để tính một đại diện của chuỗi đó. Transformer tạo ngăn xếp của các lớp tự chú ý và được giải thích dưới đây trong các phần *Scaled dot product attention* và *Multi-head attention*.

Một mô hình transformer xử lý đầu vào có kích thước thay đổi sử dụng ngăn xếp của các lớp tự chú ý thay vì [các RNN](https://www.tensorflow.org/text/tutorials/text_classification_rnn) hoặc [các CNN](https://www.tensorflow.org/tutorials/images/cnn). Kiến trúc chung này có một số ưu điểm:

* Nó không đưa ra giả định về các mối quan hệ thời gian/không gian trên dữ liệu. Đây là lý tưởng cho việc xử lý một tập các đối tượng (ví dụ, [đơn vị StarCraft](https://deepmind.com/blog/alphastar-mastering-real-time-strategy-game-starcraft-ii/#block-8)).

* Các kết quả đầu ra của lớp có thể được tính toán song song, thay vì một chuỗi như RNN.

* Các mục xa có thể ảnh hưởng đến đầu ra của nhau mà không đi qua nhiều RNN-bước, hoặc các lớp tích chập (xem ví dụ [Scene Memory Transformer](https://arxiv.org/pdf/1903.03878.pdf)).

* Nó có thể học các phụ thuộc dài hạn. Đây là một thách thức trong nhiều nhiệm vụ chuỗi.

Nhược điểm của kiến ​​trúc này là:

* Đối với một chuỗi thời gian, đầu ra cho một bước thời gian được tính từ *toàn bộ lịch sử* thay vì chỉ có đầu vào và trạng thái ẩn hiện tại. Điều này *có thể* kém hiệu quả.

* Nếu đầu vào không có một mối quan hệ thời gian/không gian, như văn bản, một số mã hóa vị trí phải được bổ sung hoặc các mô hình sẽ nhìn thấy một túi từ có hiệu quả.

Sau khi huấn luyện mô hình trong sổ tay này, bạn sẽ có thể nhập một câu tiếng Bồ Đào Nha và gửi lại bản dịch tiếng Anh.

![attention_map_portuguese](img/attention_map_portuguese.png)

## Cài đặt

In [ ]:
!pip install tensorflow_datasets
!pip install -U tensorflow-text

In [ ]:
import collections
import logging
import os
import pathlib
import re
import string
import sys
import time

import numpy as np
import matplotlib.pyplot as plt

import tensorflow_datasets as tfds
import tensorflow_text as text
import tensorflow as tf

In [ ]:
logging.getLogger('tensorflow').setLevel(logging.ERROR)  # suppress warnings

## Tải xuống tập dữ liệu

Sử dụng [tập dữ liệu TensorFlow](https://www.tensorflow.org/datasets) để tải [tập dữ liệu dịch tiếng Bồ Đào nha-Anh](https://github.com/neulab/word-embeddings-for-nmt) từ [Dự án Dịch Mở TED Talks](https://www.ted.com/participate/translate).

Tập dữ liệu này chứa khoảng 50000 ví dụ đào tạo, 1100 ví dụ xác thực và 2000 ví dụ kiểm tra.

In [ ]:
examples, metadata = tfds.load('ted_hrlr_translate/pt_to_en', with_info=True,
                               as_supervised=True)
train_examples, val_examples = examples['train'], examples['validation']

Đối tượng `tf.data.Dataset` được trả về  bởi tập dữ liệu TensorFlow các cặp ví dụ văn bản:

In [ ]:
for pt_examples, en_examples in train_examples.batch(3).take(1):
  for pt in pt_examples.numpy():
    print(pt.decode('utf-8'))

  print()

  for en in en_examples.numpy():
    print(en.decode('utf-8'))

## Token hoá văn bản & giải token hoá

Bạn không thể huấn luyện một mô hình trực tiếp trên văn bản. Trước tiên, văn bản cần được chuyển đổi thành một vài biểu diễn số. Thông thường, bạn chuyển đổi văn bản thành chuỗi token ID mà được sử dụng làm chỉ số thành một bản nhúng.

Một triển khai phổ biến được thể hiện trong [Hướng dẫn tokenizer từ con](https://www.tensorflow.org/text/guide/subwords_tokenizer) xây dựng các tokenizer từ con ([`text.BertTokenizer`](https://www.tensorflow.org/text/api_docs/python/text/BertTokenizer)) được tối ưu hóa cho tập dữ liệu này và xuất chúng trong một [saved_model](https://www.tensorflow.org/guide/saved_model).

Tải về, giải nén và nhập các `saved_model`:

In [ ]:
model_name = "ted_hrlr_translate_pt_en_converter"
tf.keras.utils.get_file(
    f"{model_name}.zip",
    f"https://storage.googleapis.com/download.tensorflow.org/models/{model_name}.zip",
    cache_dir='.', cache_subdir='', extract=True
)

In [ ]:
tokenizers = tf.saved_model.load(model_name)

Các `tf.saved_model` chứa hai tokenizer văn bản, một cho tiếng Anh và một cho tiếng Bồ Đào Nha. Cả hai đều có các phương pháp giống nhau:

In [ ]:
[item for item in dir(tokenizers.en) if not item.startswith('_')]

Phương pháp `tokenize` chuyển đổi một lô các chuỗi đến một đệm-lô các token ID. Phương pháp này tách dấu câu, chữ thường và unicode-chuẩn hóa đầu vào trước khi mã hóa. Sự chuẩn hóa đó không hiển thị ở đây vì dữ liệu đầu vào đã được chuẩn hóa.

In [ ]:
for en in en_examples.numpy():
  print(en.decode('utf-8'))

In [ ]:
encoded = tokenizers.en.tokenize(en_examples)

for row in encoded.to_list():
  print(row)

Phương pháp `detokenize` thực hiện chuyển đổi các token ID thành văn bản dễ đọc của con người:

In [ ]:
round_trip = tokenizers.en.detokenize(encoded)
for line in round_trip.numpy():
  print(line.decode('utf-8'))

Cấp thấp phương pháp `lookup` chuyển đổi từ các token ID thành token văn bản:

In [ ]:
tokens = tokenizers.en.lookup(encoded)
tokens

Ở đây bạn có thể thấy khía cạnh "subword" của các tokenizer. Từ `"searchability"` được tách thành `"search ##ability"` và từ `"serendipity"` thành `"s ##rect ##nd ##ip ##ity"`

## Cài đặt đường tin đầu vào

Để xây dựng một đường tin đầu vào phù hợp cho việc huấn luyện, bạn sẽ áp dụng một số biến đổi cho tập dữ liệu.

Hàm này sẽ được sử dụng để mã hóa các lô văn bản thô:

In [ ]:
def tokenize_pairs(pt, en):
    pt = tokenizers.pt.tokenize(pt)
    # Convert from ragged to dense, padding with zeros.
    pt = pt.to_tensor()

    en = tokenizers.en.tokenize(en)
    # Convert from ragged to dense, padding with zeros.
    en = en.to_tensor()
    return pt, en

Đây là một đường tin đầu vào đơn giản có thể xử lý, xáo trộn và phân lô dữ liệu:

In [ ]:
BUFFER_SIZE = 20000
BATCH_SIZE = 64

In [ ]:
def make_batches(ds):
  return (
      ds
      .cache()
      .shuffle(BUFFER_SIZE)
      .batch(BATCH_SIZE)
      .map(tokenize_pairs, num_parallel_calls=tf.data.AUTOTUNE)
      .prefetch(tf.data.AUTOTUNE))


train_batches = make_batches(train_examples)
val_batches = make_batches(val_examples)

## Mã hoá vị trí

Các lớp chú ý xem đầu vào của chúng là một tập hợp các vectơ, không có thứ tự tuần tự. Mô hình này cũng không chứa bất kỳ lớp đệ quy hoặc tích chập nào. Do đó, một "mã hóa vị trí" được thêm vào để cung cấp cho mô hình một số thông tin về vị trí tương đối của các token trong câu.

Vectơ mã hóa vị trí được thêm vào vectơ nhúng. Nhúng đại diện cho một token trong không gian *d* chiều nơi các token có nghĩa tương tự sẽ gần nhau hơn. Nhưng các nhúng không mã hóa vị trí tương đối của các token trong một câu. Vì vậy, sau khi thêm mã hóa vị trí, các token sẽ gần nhau hơn dựa trên *sự tương đồng nghĩa của chúng và vị trí của chúng trong câu*, trong không gian *d* chiều.

Công thức tính toán mã hóa vị trí như sau:

$$\Large{PE_{(pos, 2i)} = \sin(pos / 10000^{2i / d_{model}})}$$
$$\Large{PE_{(pos, 2i+1)} = \cos(pos / 10000^{2i / d_{model}})}$$

In [ ]:
def get_angles(pos, i, d_model):
  angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
  return pos * angle_rates

In [ ]:
def positional_encoding(position, d_model):
  angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                          np.arange(d_model)[np.newaxis, :],
                          d_model)

  # apply sin to even indices in the array; 2i
  angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

  # apply cos to odd indices in the array; 2i+1
  angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

  pos_encoding = angle_rads[np.newaxis, ...]

  return tf.cast(pos_encoding, dtype=tf.float32)

In [ ]:
n, d = 2048, 512
pos_encoding = positional_encoding(n, d)
print(pos_encoding.shape)
pos_encoding = pos_encoding[0]

# Juggle the dimensions for the plot
pos_encoding = tf.reshape(pos_encoding, (n, d//2, 2))
pos_encoding = tf.transpose(pos_encoding, (2, 1, 0))
pos_encoding = tf.reshape(pos_encoding, (d, n))

plt.pcolormesh(pos_encoding, cmap='RdBu')
plt.ylabel('Depth')
plt.xlabel('Position')
plt.colorbar()
plt.show()

## Mặt nạ

Che dấu tất cả các token pad trong lô của chuỗi. Nó đảm bảo rằng mô hình không coi padding như đầu vào. Mặt nạ chỉ ra nơi pad giá trị `0` hiện diện: nó sẽ tạo ra `1` ở những địa điểm dó, và `0` ngược lại.

In [ ]:
def create_padding_mask(seq):
  seq = tf.cast(tf.math.equal(seq, 0), tf.float32)

  # add extra dimensions to add the padding
  # to the attention logits.
  return seq[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)

In [ ]:
x = tf.constant([[7, 6, 0, 0, 1], [1, 2, 3, 0, 0], [0, 0, 0, 4, 5]])
create_padding_mask(x)

Mặt nạ nhìn trước được sử dụng để che dấu các token trong tương lai theo một chuỗi. Nói cách khác, mặt nạ cho biết mục nhập nào không nên được sử dụng.

Điều này có nghĩa là để dự đoán token thứ ba, chỉ token đầu tiên và thứ hai sẽ được sử dụng. Tương tự như vậy để dự đoán token thứ tư, chỉ token đầu tiên, thứ hai và thứ ba sẽ được sử dụng, v.v.

In [ ]:
def create_look_ahead_mask(size):
  mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
  return mask  # (seq_len, seq_len)

In [ ]:
x = tf.random.uniform((1, 3))
temp = create_look_ahead_mask(x.shape[1])
temp

## Sự chú ý tích vô hương theo tỷ lệ

![scaled_attention](img/scaled_attention.png)

Hàm chú ý được dùng bởi transformer lấy 3 đầu vào: Q (query), K (key), V (value). Phương trình được sử dụng để tính toán các trọng số chú ý là:

$$\Large{Attention(Q, K, V) = softmax_k\left(\frac{QK^T}{\sqrt{d_k}}\right) V} $$

Sự chú ý tích vô hướng được chia tỷ lệ bởi một hệ số của căn bậc 2 của độ sâu. Điều này được thực hiện bởi vì các giá trị lớn của độ sâu, tích vô hướng lớn dần theo độ lớn đẩy hàm softmax nơi mà nó có các độ dốc nhỏ dẫn đến một softmax rất khó.

Ví dụ, xem xét `Q` và `K` có trung bình là 0 và phương sai là 1. Phép nhân ma trận của chúng sẽ có trung bình 0 và phương sai `dk`. Vì vậy *căn bậc hai của* `dk` được dùng cho chia tỷ lệ, vì vậy bạn có một phương sai phù hợp không phụ thuộc vào giá trị của `dk`. Nếu phương sai quá nhỏ, đầu ra có thể là quá phẳng để tối ưu hiệu quả. Nếu phương sai quá lớn, softmax có thể bảo hoà ở khởi tạo làm nó khó khăn khi học.

Mặt nạ được nhân với -1e9 (gần âm vô cùng). Điều này được thực hiện bởi vì mặt nạ được tính tổng với phép nhân ma trận tỷ lệ của Q và K và được áp dụng ngay trước một softmax. Mục tiêu là giảm về 0 (loại bỏ) các ô này, và các đầu vào âm lớn cho softmax là gần không ở đầu ra.

In [ ]:
def scaled_dot_product_attention(q, k, v, mask):
  """Calculate the attention weights.
  q, k, v must have matching leading dimensions.
  k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v.
  The mask has different shapes depending on its type(padding or look ahead)
  but it must be broadcastable for addition.

  Args:
    q: query shape == (..., seq_len_q, depth)
    k: key shape == (..., seq_len_k, depth)
    v: value shape == (..., seq_len_v, depth_v)
    mask: Float tensor with shape broadcastable
          to (..., seq_len_q, seq_len_k). Defaults to None.

  Returns:
    output, attention_weights
  """

  matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)

  # scale matmul_qk
  dk = tf.cast(tf.shape(k)[-1], tf.float32)
  scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

  # add the mask to the scaled tensor.
  if mask is not None:
    scaled_attention_logits += (mask * -1e9)

  # softmax is normalized on the last axis (seq_len_k) so that the scores
  # add up to 1.
  attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  # (..., seq_len_q, seq_len_k)

  output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

  return output, attention_weights

Khi quá trình chuẩn hóa softmax được thực hiện trên K, các giá trị của nó quyết định mức độ quan trọng của Q.

Đầu ra đại diện cho phép nhân của trọng số chú ý và véctơ V (giá trị). Điều này đảm bảo rằng các token bạn muốn tập trung vào được giữ nguyên trạng và các token không liên quan sẽ bị loại bỏ.

In [ ]:
def print_out(q, k, v):
  temp_out, temp_attn = scaled_dot_product_attention(
      q, k, v, None)
  print('Attention weights are:')
  print(temp_attn)
  print('Output is:')
  print(temp_out)

In [ ]:
np.set_printoptions(suppress=True)

temp_k = tf.constant([[10, 0, 0],
                      [0, 10, 0],
                      [0, 0, 10],
                      [0, 0, 10]], dtype=tf.float32)  # (4, 3)

temp_v = tf.constant([[1, 0],
                      [10, 0],
                      [100, 5],
                      [1000, 6]], dtype=tf.float32)  # (4, 2)

# This `query` aligns with the second `key`,
# so the second `value` is returned.
temp_q = tf.constant([[0, 10, 0]], dtype=tf.float32)  # (1, 3)
print_out(temp_q, temp_k, temp_v)

In [ ]:
# This query aligns with a repeated key (third and fourth),
# so all associated values get averaged.
temp_q = tf.constant([[0, 0, 10]], dtype=tf.float32)  # (1, 3)
print_out(temp_q, temp_k, temp_v)

In [ ]:
# This query aligns equally with the first and second key,
# so their values get averaged.
temp_q = tf.constant([[10, 10, 0]], dtype=tf.float32)  # (1, 3)
print_out(temp_q, temp_k, temp_v)

Truyền tất cả các truy vấn cùng lúc.

In [ ]:
temp_q = tf.constant([[0, 0, 10],
                      [0, 10, 0],
                      [10, 10, 0]], dtype=tf.float32)  # (3, 3)
print_out(temp_q, temp_k, temp_v)

## Sự chú ý từ nhiều phía

![multi_head_attention](img/multi_head_attention.png)

Sự chú ý từ nhiều phía bao gồm bốn phần:

*    Các lớp tuyến tính.

*    Sự chú ý nhân vô hướng theo tỷ lệ.

*    Lớp tuyến tính cuối cùng.

Mỗi khối chú ý nhiều phía nhận được ba đầu vào: Q (truy vấn), K (khóa), V (giá trị). Chúng được đưa qua các lớp tuyến tính (Dense) trước hàm chú ý nhiều phía.

Trong sơ đồ trên `(K, Q, V)` được truyền qua các lớp tuyến tính riêng lẽ (`Dense`) cho mỗi phía chú ý. Để đơn giản/hiệu quả vào mã bên dưới triển khai cái này sử dụng một lớp dense duy nhất với `num_heads` lần như nhiều kết quả đầu ra. Kết quả được sắp xếp lại để một hình dạng của `(batch, num_heads, ...)` trước khi áp dụng hàm chú ý.

Chức năng `scaled_dot_product_attention` được định nghĩa ở trên được áp dụng trong một lời gọi duy nhất, phát sóng cho hiệu quả. Trong bước chú ý phải sử dụng một mặt nạ thích hợp. Kết quả sự chú ý cho mỗi phía sau đó được nối (sử dụng `tf.transpose`, và `tf.reshape`) và đưa qua một lớp Dense cuối cùng.

Thay vì một phía chú ý duy nhất, Q, K và V được chia thành nhiều phía vì nó cho phép mô hình cùng tham gia vào thông tin từ các không gian con biểu diễn khác nhau ở các vị trí khác nhau. Sau khi tách, mỗi phía có số chiều được giảm, do đó, tổng chi phí tính toán giống như sự chú ý của phía duy nhất với kích thước đầy đủ.

In [ ]:
class MultiHeadAttention(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads):
    super(MultiHeadAttention, self).__init__()
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0

    self.depth = d_model // self.num_heads

    self.wq = tf.keras.layers.Dense(d_model)
    self.wk = tf.keras.layers.Dense(d_model)
    self.wv = tf.keras.layers.Dense(d_model)

    self.dense = tf.keras.layers.Dense(d_model)

  def split_heads(self, x, batch_size):
    """Split the last dimension into (num_heads, depth).
    Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
    """
    x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(x, perm=[0, 2, 1, 3])

  def call(self, v, k, q, mask):
    batch_size = tf.shape(q)[0]

    q = self.wq(q)  # (batch_size, seq_len, d_model)
    k = self.wk(k)  # (batch_size, seq_len, d_model)
    v = self.wv(v)  # (batch_size, seq_len, d_model)

    q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
    k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
    v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)

    # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
    # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
    scaled_attention, attention_weights = scaled_dot_product_attention(
        q, k, v, mask)

    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

    concat_attention = tf.reshape(scaled_attention,
                                  (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

    output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)

    return output, attention_weights

Tạo một lớp `MultiHeadAttention` để thử kỹ càng. Tại mỗi vị trí trong chuỗi, `y`, `MultiHeadAttention` chạy hết 8 phía chú ý trên tất cả các vị trí khác nhau trong chuỗi, trả về một véctơ với cùng chiều dài ở mỗi vị trí.

In [ ]:
temp_mha = MultiHeadAttention(d_model=512, num_heads=8)
y = tf.random.uniform((1, 60, 512))  # (batch_size, encoder_sequence, d_model)
out, attn = temp_mha(y, k=y, q=y, mask=None)
out.shape, attn.shape

## Mạng truyền thẳng điểm định tính

Mạng truyền thẳng định tính bao gồm hai lớp kết nối đầy đủ với một kích hoạt ReLU ở giữa.

In [ ]:
def point_wise_feed_forward_network(d_model, dff):
  return tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),  # (batch_size, seq_len, dff)
      tf.keras.layers.Dense(d_model)  # (batch_size, seq_len, d_model)
  ])

In [ ]:
sample_ffn = point_wise_feed_forward_network(512, 2048)
sample_ffn(tf.random.uniform((64, 50, 512))).shape

## Mã hoá và giải mã

![transformer](img/transformer.png)

Mô hình transformer theo sau cùng mẫu chung như một [chuối sang chuỗi với mô hình đáng chú ý](https://www.tensorflow.org/text/tutorials/nmt_with_attention.ipynb) tiêu chuẩn. 

* Câu đầu vào được chuyển qua `N` lớp mã hoá tạo một đầu ra cho mỗi token trong chuỗi.

* Giải mã tham gia vào đầu ra của mã hoá với đầu vào của nó (tự chú ý) để dự đoán từ tiếp theo.

### Lớp mã hóa

Mỗi lớp mã hóa bao gồm các lớp con:

1. Chú ý nhiều phía (với mặt nạ đệm)

2. Các mạng truyền thẳng định tính

Mỗi lớp con này có một kết nối dư xung quanh nó, theo sau là chuẩn hóa lớp. Các kết nối dư giúp tránh vấn đề độ dốc biến mất trong các mạng sâu.

Đầu ra của mỗi lớp con là `LayerNorm(x + Sublayer(x))`. Chuẩn hóa được thực hiện trên trục `d_model` (cuối cùng). Có N lớp mã hóa trong transformer.

In [ ]:
class EncoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, dff, rate=0.1):
    super(EncoderLayer, self).__init__()

    self.mha = MultiHeadAttention(d_model, num_heads)
    self.ffn = point_wise_feed_forward_network(d_model, dff)

    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

    self.dropout1 = tf.keras.layers.Dropout(rate)
    self.dropout2 = tf.keras.layers.Dropout(rate)

  def call(self, x, training, mask):

    attn_output, _ = self.mha(x, x, x, mask)  # (batch_size, input_seq_len, d_model)
    attn_output = self.dropout1(attn_output, training=training)
    out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)

    ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
    ffn_output = self.dropout2(ffn_output, training=training)
    out2 = self.layernorm2(out1 + ffn_output)  # (batch_size, input_seq_len, d_model)

    return out2

In [ ]:
sample_encoder_layer = EncoderLayer(512, 8, 2048)

sample_encoder_layer_output = sample_encoder_layer(
    tf.random.uniform((64, 43, 512)), False, None)

sample_encoder_layer_output.shape  # (batch_size, input_seq_len, d_model)

### Lớp giải mã

Mỗi lớp giải mã bao gồm các lớp con:

1. Mặt nạ chú ý nhiều phía (với mặt nạ nhìn trước và mặt nạ đệm)

2. Chú ý nhiều phía (có mặt nạ đệm). V (giá trị) và K (key) nhận được *đầu ra bộ mã hóa* là các đầu vào. Q (query) nhận được *đầu ra từ lớp con chú ý nhiều phía được che giấu*.

3. Các mạng truyền thẳng định tính

Mỗi lớp con này có một kết nối dư xung quanh nó, theo sau là chuẩn hóa lớp. Đầu ra của mỗi lớp con là `LayerNorm(x + Sublayer(x))`. Chuẩn hóa được thực hiện trên trục `d_model` (cuối cùng).

Có N lớp giải mã trong transformer.

Khi Q nhận đầu ra từ khối chú ý đầu tiên của bộ giải mã và K nhận đầu ra của bộ mã hóa, trọng số chú ý thể hiện sự quan trọng đối với đầu vào của bộ giải mã dựa trên đầu ra của bộ mã hóa. Nói cách khác, bộ giải mã dự đoán token tiếp theo bằng cách xem đầu ra của bộ mã hóa và tự tham gia vào đầu ra của chính nó. Xem phần minh họa ở trên trong phần *sự chú ý tích vô phương theo tỷ lệ*.

In [ ]:
class DecoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, dff, rate=0.1):
    super(DecoderLayer, self).__init__()

    self.mha1 = MultiHeadAttention(d_model, num_heads)
    self.mha2 = MultiHeadAttention(d_model, num_heads)

    self.ffn = point_wise_feed_forward_network(d_model, dff)

    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

    self.dropout1 = tf.keras.layers.Dropout(rate)
    self.dropout2 = tf.keras.layers.Dropout(rate)
    self.dropout3 = tf.keras.layers.Dropout(rate)

  def call(self, x, enc_output, training,
           look_ahead_mask, padding_mask):
    # enc_output.shape == (batch_size, input_seq_len, d_model)

    attn1, attn_weights_block1 = self.mha1(x, x, x, look_ahead_mask)  # (batch_size, target_seq_len, d_model)
    attn1 = self.dropout1(attn1, training=training)
    out1 = self.layernorm1(attn1 + x)

    attn2, attn_weights_block2 = self.mha2(
        enc_output, enc_output, out1, padding_mask)  # (batch_size, target_seq_len, d_model)
    attn2 = self.dropout2(attn2, training=training)
    out2 = self.layernorm2(attn2 + out1)  # (batch_size, target_seq_len, d_model)

    ffn_output = self.ffn(out2)  # (batch_size, target_seq_len, d_model)
    ffn_output = self.dropout3(ffn_output, training=training)
    out3 = self.layernorm3(ffn_output + out2)  # (batch_size, target_seq_len, d_model)

    return out3, attn_weights_block1, attn_weights_block2

In [ ]:
sample_decoder_layer = DecoderLayer(512, 8, 2048)

sample_decoder_layer_output, _, _ = sample_decoder_layer(
    tf.random.uniform((64, 50, 512)), sample_encoder_layer_output,
    False, None, None)

sample_decoder_layer_output.shape  # (batch_size, target_seq_len, d_model)

### Mã hoá

`Encoder` bao gồm:

1. Nhúng đầu vào

2. Mã hóa vị trí

3. N lớp mã hóa

Đầu vào được đưa qua một phép nhúng được tổng hợp bằng mã hóa vị trí. Đầu ra của tổng hợp này là đầu vào cho các lớp mã hóa. Đầu ra của bộ mã hóa là đầu vào cho bộ giải mã.

In [ ]:
class Encoder(tf.keras.layers.Layer):
  def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size,
               maximum_position_encoding, rate=0.1):
    super(Encoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    self.embedding = tf.keras.layers.Embedding(input_vocab_size, d_model)
    self.pos_encoding = positional_encoding(maximum_position_encoding,
                                            self.d_model)

    self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate)
                       for _ in range(num_layers)]

    self.dropout = tf.keras.layers.Dropout(rate)

  def call(self, x, training, mask):

    seq_len = tf.shape(x)[1]

    # adding embedding and position encoding.
    x = self.embedding(x)  # (batch_size, input_seq_len, d_model)
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x += self.pos_encoding[:, :seq_len, :]

    x = self.dropout(x, training=training)

    for i in range(self.num_layers):
      x = self.enc_layers[i](x, training, mask)

    return x  # (batch_size, input_seq_len, d_model)

In [ ]:
sample_encoder = Encoder(num_layers=2, d_model=512, num_heads=8,
                         dff=2048, input_vocab_size=8500,
                         maximum_position_encoding=10000)
temp_input = tf.random.uniform((64, 62), dtype=tf.int64, minval=0, maxval=200)

sample_encoder_output = sample_encoder(temp_input, training=False, mask=None)

print(sample_encoder_output.shape)  # (batch_size, input_seq_len, d_model)

### Giải mã

`Decoder` bao gồm:

1. Nhúng đầu ra

2. Mã hóa vị trí

3. N lớp giải mã

Mục tiêu được đưa qua một phép nhúng được tổng hợp bằng mã hóa vị trí. Đầu ra của phép tổng hợp này là đầu vào cho các lớp giải mã. Đầu ra của bộ giải mã là đầu vào của lớp tuyến tính cuối cùng.

In [ ]:
class Decoder(tf.keras.layers.Layer):
  def __init__(self, num_layers, d_model, num_heads, dff, target_vocab_size,
               maximum_position_encoding, rate=0.1):
    super(Decoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    self.embedding = tf.keras.layers.Embedding(target_vocab_size, d_model)
    self.pos_encoding = positional_encoding(maximum_position_encoding, d_model)

    self.dec_layers = [DecoderLayer(d_model, num_heads, dff, rate)
                       for _ in range(num_layers)]
    self.dropout = tf.keras.layers.Dropout(rate)

  def call(self, x, enc_output, training,
           look_ahead_mask, padding_mask):

    seq_len = tf.shape(x)[1]
    attention_weights = {}

    x = self.embedding(x)  # (batch_size, target_seq_len, d_model)
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x += self.pos_encoding[:, :seq_len, :]

    x = self.dropout(x, training=training)

    for i in range(self.num_layers):
      x, block1, block2 = self.dec_layers[i](x, enc_output, training,
                                             look_ahead_mask, padding_mask)

      attention_weights[f'decoder_layer{i+1}_block1'] = block1
      attention_weights[f'decoder_layer{i+1}_block2'] = block2

    # x.shape == (batch_size, target_seq_len, d_model)
    return x, attention_weights

In [ ]:
sample_decoder = Decoder(num_layers=2, d_model=512, num_heads=8,
                         dff=2048, target_vocab_size=8000,
                         maximum_position_encoding=5000)
temp_input = tf.random.uniform((64, 26), dtype=tf.int64, minval=0, maxval=200)

output, attn = sample_decoder(temp_input,
                              enc_output=sample_encoder_output,
                              training=False,
                              look_ahead_mask=None,
                              padding_mask=None)

output.shape, attn['decoder_layer2_block2'].shape

## Tạo Transformer

Transformer bao gồm bộ mã hóa, bộ giải mã và một lớp tuyến tính cuối cùng. Đầu ra của bộ giải mã là đầu vào của lớp tuyến tính và đầu ra của nó được trả về.

In [ ]:
class Transformer(tf.keras.Model):
  def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size,
               target_vocab_size, pe_input, pe_target, rate=0.1):
    super().__init__()
    self.encoder = Encoder(num_layers, d_model, num_heads, dff,
                             input_vocab_size, pe_input, rate)

    self.decoder = Decoder(num_layers, d_model, num_heads, dff,
                           target_vocab_size, pe_target, rate)

    self.final_layer = tf.keras.layers.Dense(target_vocab_size)

  def call(self, inputs, training):
    # Keras models prefer if you pass all your inputs in the first argument
    inp, tar = inputs

    enc_padding_mask, look_ahead_mask, dec_padding_mask = self.create_masks(inp, tar)

    enc_output = self.encoder(inp, training, enc_padding_mask)  # (batch_size, inp_seq_len, d_model)

    # dec_output.shape == (batch_size, tar_seq_len, d_model)
    dec_output, attention_weights = self.decoder(
        tar, enc_output, training, look_ahead_mask, dec_padding_mask)

    final_output = self.final_layer(dec_output)  # (batch_size, tar_seq_len, target_vocab_size)

    return final_output, attention_weights

  def create_masks(self, inp, tar):
    # Encoder padding mask
    enc_padding_mask = create_padding_mask(inp)

    # Used in the 2nd attention block in the decoder.
    # This padding mask is used to mask the encoder outputs.
    dec_padding_mask = create_padding_mask(inp)

    # Used in the 1st attention block in the decoder.
    # It is used to pad and mask future tokens in the input received by
    # the decoder.
    look_ahead_mask = create_look_ahead_mask(tf.shape(tar)[1])
    dec_target_padding_mask = create_padding_mask(tar)
    look_ahead_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)

    return enc_padding_mask, look_ahead_mask, dec_padding_mask

In [ ]:
sample_transformer = Transformer(
    num_layers=2, d_model=512, num_heads=8, dff=2048,
    input_vocab_size=8500, target_vocab_size=8000,
    pe_input=10000, pe_target=6000)

temp_input = tf.random.uniform((64, 38), dtype=tf.int64, minval=0, maxval=200)
temp_target = tf.random.uniform((64, 36), dtype=tf.int64, minval=0, maxval=200)

fn_out, _ = sample_transformer([temp_input, temp_target], training=False)

fn_out.shape  # (batch_size, tar_seq_len, target_vocab_size)

## Đặt siêu tham số

Để giữ cho ví dụ này nhỏ và tương đối nhanh, các giá trị cho `num_layers, d_model, dff` đã được giảm.

Mô hình cơ sở được mô tả trong [bài](https://arxiv.org/abs/1706.03762) sử dụng: `num_layers=6, d_model=512, dff=2048`.

In [ ]:
num_layers = 4
d_model = 128
dff = 512
num_heads = 8
dropout_rate = 0.1

## Bộ tối ưu hoá

Sử dụng bộ tối ưu Adam với một bộ lên lịch  tỷ lệ học tùy chỉnh theo công thức trong [bài](https://arxiv.org/abs/1706.03762).

$$\Large{lrate = d_{model}^{-0.5} * \min(step{\_}num^{-0.5}, step{\_}num \cdot warmup{\_}steps^{-1.5})}$$

In [ ]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, d_model, warmup_steps=4000):
    super(CustomSchedule, self).__init__()

    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps

  def __call__(self, step):
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps ** -1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [ ]:
learning_rate = CustomSchedule(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98,
                                     epsilon=1e-9)

In [ ]:
temp_learning_rate_schedule = CustomSchedule(d_model)

plt.plot(temp_learning_rate_schedule(tf.range(40000, dtype=tf.float32)))
plt.ylabel("Learning Rate")
plt.xlabel("Train Step")

## Mất mát và chỉ số đo lường

Vì các chuỗi mục tiêu được đệm, điều quan trọng là phải áp dụng mặt nạ đệm khi tính toán mất mát.

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

In [ ]:
def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_sum(loss_)/tf.reduce_sum(mask)


def accuracy_function(real, pred):
  accuracies = tf.equal(real, tf.argmax(pred, axis=2))

  mask = tf.math.logical_not(tf.math.equal(real, 0))
  accuracies = tf.math.logical_and(mask, accuracies)

  accuracies = tf.cast(accuracies, dtype=tf.float32)
  mask = tf.cast(mask, dtype=tf.float32)
  return tf.reduce_sum(accuracies)/tf.reduce_sum(mask)

In [ ]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.Mean(name='train_accuracy')

## Huấn luyện và điểm kiểm tra

In [ ]:
transformer = Transformer(
    num_layers=num_layers,
    d_model=d_model,
    num_heads=num_heads,
    dff=dff,
    input_vocab_size=tokenizers.pt.get_vocab_size().numpy(),
    target_vocab_size=tokenizers.en.get_vocab_size().numpy(),
    pe_input=1000,
    pe_target=1000,
    rate=dropout_rate)

Tạo đường dẫn điểm kiểm tra và trình quản lý điểm kiểm tra. Điều này sẽ được sử dụng để lưu các điểm kiểm tra mỗi `n` epoch.

In [ ]:
checkpoint_path = "./checkpoints/train"

ckpt = tf.train.Checkpoint(transformer=transformer,
                           optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

# if a checkpoint exists, restore the latest checkpoint.
if ckpt_manager.latest_checkpoint:
  ckpt.restore(ckpt_manager.latest_checkpoint)
  print('Latest checkpoint restored!!')

Mục tiêu được chia thành `tar_inp` và `tar_real`. `tar_inp` được chuyển đi như một đầu vào cho bộ giải mã. `tar_real` là cùng một đầu vào được dịch chuyển bởi 1: Tại mỗi vị trí trong `tar_input`, `tar_real` chứa token kế tiếp cần được dự đoán.

Ví dụ, `sentence` = "SOS A lion in the jungle is sleeping EOS"

`tar_inp` =  "SOS A lion in the jungle is sleeping"

`tar_real` = "A lion in the jungle is sleeping EOS"

Transformer là một mô hình tự động hồi quy: nó đưa ra dự đoán từng phần một và sử dụng kết quả đầu ra của nó để quyết định phải làm gì tiếp theo.

Trong huấn luyện ví dụ này sử dụng teacher-forcing (như trong hướng dẫn [sinh văn bản](https://www.tensorflow.org/text/tutorials/text_generation)). Giáo viên buộc phải chuyển đầu ra thực sự cho bước thời gian tiếp theo bất kể mô hình dự đoán những gì ở bước thời gian hiện tại.

Khi transformer dự đoán mỗi token, *tự chú ý* cho phép nó để nhìn vào các token trước đó trong chuỗi đầu vào tốt hơn dự đoán token tiếp theo.

Để ngăn mô hình nhìn trộm đầu ra dự kiến, mô hình sử dụng mặt nạ nhìn trước.

In [ ]:
EPOCHS = 20

In [ ]:
# The @tf.function trace-compiles train_step into a TF graph for faster
# execution. The function specializes to the precise shape of the argument
# tensors. To avoid re-tracing due to the variable sequence lengths or variable
# batch sizes (the last batch is smaller), use input_signature to specify
# more generic shapes.

train_step_signature = [
    tf.TensorSpec(shape=(None, None), dtype=tf.int64),
    tf.TensorSpec(shape=(None, None), dtype=tf.int64),
]


@tf.function(input_signature=train_step_signature)
def train_step(inp, tar):
  tar_inp = tar[:, :-1]
  tar_real = tar[:, 1:]

  with tf.GradientTape() as tape:
    predictions, _ = transformer([inp, tar_inp],
                                 training = True)
    loss = loss_function(tar_real, predictions)

  gradients = tape.gradient(loss, transformer.trainable_variables)
  optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))

  train_loss(loss)
  train_accuracy(accuracy_function(tar_real, predictions))

Tiếng Bồ Đào Nha được sử dụng làm ngôn ngữ đầu vào và tiếng Anh là ngôn ngữ đích.

In [ ]:
for epoch in range(EPOCHS):
  start = time.time()

  train_loss.reset_states()
  train_accuracy.reset_states()

  # inp -> portuguese, tar -> english
  for (batch, (inp, tar)) in enumerate(train_batches):
    train_step(inp, tar)

    if batch % 50 == 0:
      print(f'Epoch {epoch + 1} Batch {batch} Loss {train_loss.result():.4f} Accuracy {train_accuracy.result():.4f}')

  if (epoch + 1) % 5 == 0:
    ckpt_save_path = ckpt_manager.save()
    print(f'Saving checkpoint for epoch {epoch+1} at {ckpt_save_path}')

  print(f'Epoch {epoch + 1} Loss {train_loss.result():.4f} Accuracy {train_accuracy.result():.4f}')

  print(f'Time taken for 1 epoch: {time.time() - start:.2f} secs\n')

### Chạy suy luận

Các bước sau được sử dụng để suy luận:

* Mã hóa các câu đầu vào bằng cách sử dụng tokenizer Bồ Đào Nha (`tokenizers.pt`). Đây là đầu vào của bộ mã hóa.

* Các đầu vào bộ giải mã được khởi tạo với token `[START]`.

* Tính toán mặt nạ đệm và mặt nạ nhìn trước.

* Các `decoder` sau đó các kết quả đầu ra dự đoán bằng cách nhìn vào `encoder output` và đầu ra riêng (tự chú ý) của nó.

* Nối token dự đoán với đầu vào của bộ giải mã và chuyển nó đến bộ giải mã.

* Trong cách tiếp cận này, bộ giải mã dự đoán token tiếp theo dựa trên các token trước đó nó đã dự đoán.

Lưu ý: Mô hình này được tối ưu hóa cho *hiệu quả huấn luyện* và làm cho một dự đoán tiếp theo cho mỗi *token* trong đầu ra cùng một lúc. Điều này là thừa trong quá trình suy luận và chỉ dự đoán cuối cùng được sử dụng. Mô hình này có thể được thực hiện hiệu quả hơn đối với kết luận nếu bạn chỉ tính toán dự đoán trước khi chạy ở chế độ suy luận (`training=False`).

In [ ]:
class Translator(tf.Module):
  def __init__(self, tokenizers, transformer):
    self.tokenizers = tokenizers
    self.transformer = transformer

  def __call__(self, sentence, max_length=20):
    # input sentence is portuguese, hence adding the start and end token
    assert isinstance(sentence, tf.Tensor)
    if len(sentence.shape) == 0:
      sentence = sentence[tf.newaxis]

    sentence = self.tokenizers.pt.tokenize(sentence).to_tensor()
    
    encoder_input = sentence

    # as the target is english, the first token to the transformer should be the
    # english start token.
    start_end = self.tokenizers.en.tokenize([''])[0]
    start = start_end[0][tf.newaxis]
    end = start_end[1][tf.newaxis]

    # `tf.TensorArray` is required here (instead of a python list) so that the
    # dynamic-loop can be traced by `tf.function`.
    output_array = tf.TensorArray(dtype=tf.int64, size=0, dynamic_size=True)
    output_array = output_array.write(0, start)
    
    for i in tf.range(max_length):
      output = tf.transpose(output_array.stack())
      predictions, _ = self.transformer([encoder_input, output], training=False)
      
      # select the last token from the seq_len dimension
      predictions = predictions[:, -1:, :]  # (batch_size, 1, vocab_size)

      predicted_id = tf.argmax(predictions, axis=-1)

      # concatentate the predicted_id to the output which is given to the decoder
      # as its input.
      output_array = output_array.write(i+1, predicted_id[0])

      if predicted_id == end:
        break

    output = tf.transpose(output_array.stack())
    # output.shape (1, tokens)
    text = tokenizers.en.detokenize(output)[0]  # shape: ()

    tokens = tokenizers.en.lookup(output)[0]

    # `tf.function` prevents us from using the attention_weights that were
    # calculated on the last iteration of the loop. So recalculate them outside
    # the loop.
    _, attention_weights = self.transformer([encoder_input, output[:,:-1]], training=False)

    return text, tokens, attention_weights

Tạo một thể hiện của này lớp `Translator`, và thử nó ra một vài lần:

In [ ]:
translator = Translator(tokenizers, transformer)

In [ ]:
def print_translation(sentence, tokens, ground_truth):
  print(f'{"Input:":15s}: {sentence}')
  print(f'{"Prediction":15s}: {tokens.numpy().decode("utf-8")}')
  print(f'{"Ground truth":15s}: {ground_truth}')

In [ ]:
sentence = "este é um problema que temos que resolver."
ground_truth = "this is a problem we have to solve ."

translated_text, translated_tokens, attention_weights = translator(
    tf.constant(sentence))
print_translation(sentence, translated_text, ground_truth)

In [ ]:
sentence = "os meus vizinhos ouviram sobre esta ideia."
ground_truth = "and my neighboring homes heard about this idea ."

translated_text, translated_tokens, attention_weights = translator(
    tf.constant(sentence))
print_translation(sentence, translated_text, ground_truth)

In [ ]:
sentence = "vou então muito rapidamente partilhar convosco algumas histórias de algumas coisas mágicas que aconteceram."
ground_truth = "so i \'ll just share with you some stories very quickly of some magical things that have happened ."

translated_text, translated_tokens, attention_weights = translator(
    tf.constant(sentence))
print_translation(sentence, translated_text, ground_truth)

## Vẽ đồ thị sự chú ý

Lớp `Translator` trả về một cuốn từ điển của các bản đồ chú ý bản đồ mà bạn có thể sử dụng để hình dung làm việc nội bộ của mô hình:

The `Translator` class returns a dictionary of attention maps you can use to visualize the internal working of the model:

In [ ]:
sentence = "este é o primeiro livro que eu fiz."
ground_truth = "this is the first book i've ever done."

translated_text, translated_tokens, attention_weights = translator(
    tf.constant(sentence))
print_translation(sentence, translated_text, ground_truth)

In [ ]:
def plot_attention_head(in_tokens, translated_tokens, attention):
  # The plot is of the attention when a token was generated.
  # The model didn't generate `<START>` in the output. Skip it.
  translated_tokens = translated_tokens[1:]

  ax = plt.gca()
  ax.matshow(attention)
  ax.set_xticks(range(len(in_tokens)))
  ax.set_yticks(range(len(translated_tokens)))

  labels = [label.decode('utf-8') for label in in_tokens.numpy()]
  ax.set_xticklabels(
      labels, rotation=90)

  labels = [label.decode('utf-8') for label in translated_tokens.numpy()]
  ax.set_yticklabels(labels)

In [ ]:
head = 0
# shape: (batch=1, num_heads, seq_len_q, seq_len_k)
attention_heads = tf.squeeze(
  attention_weights['decoder_layer4_block2'], 0)
attention = attention_heads[head]
attention.shape

In [ ]:
in_tokens = tf.convert_to_tensor([sentence])
in_tokens = tokenizers.pt.tokenize(in_tokens).to_tensor()
in_tokens = tokenizers.pt.lookup(in_tokens)[0]
in_tokens

In [ ]:
translated_tokens

In [ ]:
plot_attention_head(in_tokens, translated_tokens, attention)

In [ ]:
def plot_attention_weights(sentence, translated_tokens, attention_heads):
  in_tokens = tf.convert_to_tensor([sentence])
  in_tokens = tokenizers.pt.tokenize(in_tokens).to_tensor()
  in_tokens = tokenizers.pt.lookup(in_tokens)[0]
  in_tokens

  fig = plt.figure(figsize=(16, 8))

  for h, head in enumerate(attention_heads):
    ax = fig.add_subplot(2, 4, h+1)

    plot_attention_head(in_tokens, translated_tokens, head)

    ax.set_xlabel(f'Head {h+1}')

  plt.tight_layout()
  plt.show()

In [ ]:
plot_attention_weights(sentence, translated_tokens,
                       attention_weights['decoder_layer4_block2'][0])

Mô hình không ổn đối với các từ không quen thuộc. Cả `"triceratops"` hoặc `"encyclopedia"` đều không có trong tập dữ liệu đầu vào và mô hình gần như học cách chuyển ngữ chúng, ngay cả khi không có từ vựng được chia sẻ:

In [ ]:
sentence = "Eu li sobre triceratops na enciclopédia."
ground_truth = "I read about triceratops in the encyclopedia."

translated_text, translated_tokens, attention_weights = translator(
    tf.constant(sentence))
print_translation(sentence, translated_text, ground_truth)

plot_attention_weights(sentence, translated_tokens,
                       attention_weights['decoder_layer4_block2'][0])

## Xuất

Đó là mô hình suy luận đang làm việc, vì vậy tiếp theo bạn sẽ xuất nó như là một `tf.saved_model`.

Để làm điều đó, gói nó trong một lớp con `tf.Module`, lần này với một `tf.function` trên phương thức `__call__`:

In [ ]:
class ExportTranslator(tf.Module):
  def __init__(self, translator):
    self.translator = translator

  @tf.function(input_signature=[tf.TensorSpec(shape=[], dtype=tf.string)])
  def __call__(self, sentence):
    (result, 
     tokens,
     attention_weights) = self.translator(sentence, max_length=100)
    
    return result

Ở `tf.function` phía trên chỉ có câu đầu ra được trả về. Nhờ có sự [thực thi không nghiêm ngặt](https://tensorflow.org/guide/intro_to_graphs) trong `tf.function` bất kỳ giá trị không cần thiết không bao giờ tính toán.

In [ ]:
translator = ExportTranslator(translator)

Since the model is decoding the predictions using `tf.argmax` the predictions are deterministic. The original model and one reloaded from its `SavedModel` should give identical predictions:

In [ ]:
translator("este é o primeiro livro que eu fiz.").numpy()

In [ ]:
tf.saved_model.save(translator, export_dir='translator')

In [ ]:
reloaded = tf.saved_model.load('translator')

In [ ]:
reloaded("este é o primeiro livro que eu fiz.").numpy()

## Tóm tắt

Trong hướng dẫn này, bạn đã học về mã hóa vị trí, sự chú ý nhiều phía, tầm quan trọng của mặt nạ và cách tạo một transformer.

Hãy thử sử dụng một tập dữ liệu khác để huấn luyện transformer. Bạn cũng có thể tạo transformer cơ sở hoặc transformer XL bằng cách thay đổi các siêu tham số ở trên. Bạn cũng có thể sử dụng các lớp định nghĩa ở đây để tạo ra [Bert](https://arxiv.org/abs/1810.04805) và huấn luyện các mô hình hiện đại nhất. Hơn nữa, bạn có thể triển khai tìm kiếm beam để có được những dự đoán tốt hơn.